In [133]:
import pandas as pd
import numpy as np
import time
import string
import re
from IPython.display import clear_output
import requests
import spacy
import gensim
import yake
from gensim import corpora
from nltk.corpus import stopwords
import concurrent.futures
#nltk.download("stopwords")

import warnings
warnings.filterwarnings("ignore")

In [139]:
#!python3 -m spacy download de_core_news_md
nlp = spacy.load('de_core_news_md')

---

In [2]:
data = pd.read_csv("./data/2783_excellent_article.csv")

In [3]:
data.head()

,Unnamed: 0,pageid,content
0,0,1428,"[[Datei:Danimarca XIII secolo, plinio historia..."
1,1,2677,{{Begriffsklärungshinweis}}\n{{Infobox Staat\n...
2,2,1200964,{{Dieser Artikel|behandelt den Staat. Zu weite...
3,3,880316,{{Begriffsklärungshinweis}}\n{{Infobox Staat\n...
4,4,2391,{{Dieser Artikel|behandelt das Land – zu ander...


In [144]:
text = data["content"][2]

In [145]:
text

'{{Dieser Artikel|behandelt den Staat. Zu weiteren Bedeutungen siehe [[Australien (Begriffsklärung)]].}}\n{{Infobox Staat\n|NAME-AMTSSPRACHE   = Commonwealth of Australia\n|NAME-DEUTSCH       = Australien\n|BILD-FLAGGE        = Flag of Australia (converted).svg\n|ARTIKEL-FLAGGE     = Flagge Australiens\n|BILD-WAPPEN        = Coat of Arms of Australia.svg\n|ARTIKEL-WAPPEN     = Wappen Australiens\n|WAHLSPRUCH         = \n|AMTSSPRACHE        = \'\'[[de jure]]:\'\' keine<br />\'\'[[de facto]]:\'\' [[Englische Sprache|Englisch]]<ref>{{Internetquelle |url=https://voebb.brockhaus.de/sites/default/files/pdfpermlink/australien-c72357d.pdf |titel=Australien |werk=Die Brockhaus Enzyklopädie Online |datum=2016-10-28 |format=PDF; 183&nbsp;kB |offline=1 |archiv-url=https://web.archive.org/web/20170204000205/https://voebb.brockhaus.de/sites/default/files/pdfpermlink/australien-c72357d.pdf |archiv-datum=2017-02-04 |abruf=2019-09-11 |kommentar=Original nur mit Anmeldung abrufbar}}</ref>\n|HAUPTSTADT  

In [146]:
t = re.sub(r"\n", "", text)
t = re.sub(r"\{{2}.*?\}{2}", "", t)
t = re.sub(r"\}{2}", "", t)
t = re.sub(r"\[\[[0-9A-Za-z\s()]*?\|", "", t)
t = re.sub(r"\[\[", "", t)
t = re.sub(r"\]\]", "", t)
t = re.sub(r"\[.*?\]", "", t)
t = re.sub(r"<ref.*?/ref>", "", t)
t = re.sub(r"<.*?>", "", t)
t = re.sub(r"\|", " ", t)
t = re.sub(r"\'", "", t)

# source: url_extract_pattern from https://uibakery.io/regex-library/url-regex-python
url_extract_pattern = "https?:\\/\\/(?:www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)"
t = re.sub(url_extract_pattern, '', t)

t = re.sub(r"--+", "", t)
t = re.sub(r"==+", "", t)
t = re.sub(r"\&nbsp;", "", t)
# Dateinamen
t = re.sub(r"[^ ]*\..{4}|[^ ]*\..{3}", "", t)

#source: https://stackoverflow.com/questions/2077897/substitute-multiple-whitespace-with-single-whitespace-in-python
_RE_COMBINE_WHITESPACE = re.compile(r"\s+")
t = _RE_COMBINE_WHITESPACE.sub(" ", t).strip()

In [147]:
t

'HAUPTSTADT = Canberra STAATS- UND REGIERUNGSFORM = Bundesstaat (föderaler Staat) föderale parlamentarische Monarchie STAATSOBERHAUPT = König Charles reten durch Generalgouverneur DavidHurley REGIERUNGSCHEF = PremierministerAnthony Albanese FLÄCHE = (Liste von Staaten und Territorien nach Fläche NWOHNER = 25,8 Millionen (Liste von Staaten und Territorien nach Einwohnerzahl#Liste eptember 2021) BEV-DICHTE = 3,3 BEV-ZUNAHME = +0,3 % BEV-ZUNAHME-ZEITRAUM= (September 2021)* 1,6 Billionen USD (Liste der Länder nach Bruttoinlandsprodukt ,5 Billionen USD (Liste der Länder nach Bruttoinlandsprodukt USD (Liste der Länder nach Bruttoinlandsprodukt pro Kopf USD (Liste der Länder nach Bruttoinlandsprodukt pro Kopf P-ERWEITERT = * Total (nominal)* Total (Kaufkraftparität KKP)* P) HDI = 0,951 (Index der menschlichen Entwicklung#Sehr hohe menschliche Entwicklung 021) WÄHRUNG = Australischer Dollar (AUD) GRÜNDUNG = uar 1901 UNABHÄNGIGKEIT = tember ober 1942(Statut von z 1986(Australia Act) NATIONALHYM

In [148]:
german_stopwords = stopwords.words("german")

In [161]:
t_word_list = t.split(" ")

# lemmatization
text_lemma = []

for ix, word in enumerate(t_word_list):
    doc = nlp(word)
    result = ' '.join([x.lemma_ for x in doc]) 
    text_lemma.append(result)

lemma_word_list = [gensim.utils.simple_preprocess(word, deacc = True) for word in text_lemma]

In [164]:
# remove stopwords
# source: https://stackoverflow.com/questions/5486337/how-to-remove-stop-words-using-nltk-or-python
#final = [word for word in lemma_word_list if word not in german_stopwords]

final = []
for word in lemma_word_list:
    if len(word) > 0 and word[0] not in german_stopwords:
        final.append(word[0])

In [167]:
final_text = " ".join(final)

In [170]:
kw_extractor = yake.KeywordExtractor()
keywords = kw_extractor.extract_keywords(t)

for kw in keywords:
    print(kw)

('Advance Australia Fair', 6.602695987900342e-05)
('Seefahrer Willem Jansz', 0.00012936277882499516)
('Milingimbi Bevölkerungszahl Australiens', 0.00019269149488840263)
('Leader Anthony Albanese', 0.00019757464525809715)
('niederländische Seefahrer Frederick', 0.0002232541662248534)
('der Westküste Australiens', 0.00023180273319391625)
('Swan River Colony', 0.00024100659166373896)
('der', 0.0002500544094203898)
('der Erde Flagge', 0.00025088071681615476)
('Westküste Australiens von', 0.000277169131472566)
('Niederländers Willem Jansz', 0.00028116014188124257)
('Entdeckers Abel Tasman', 0.00028413316053539596)
('Australien', 0.00030425805899504606)
('die', 0.00031979657051419476)
('der Entdeckung Australiens', 0.00032942101302263304)
('Entdeckungsreisenden Matthew Flinders', 0.00034027525840081693)
('Holländer Abel Tasman', 0.00034027525840081693)
('niederländische Seefahrer die', 0.00034683233671754847)
('Flagge der Eureka', 0.00035801121488444415)
('der niederländische Seefahrer', 0.0

In [171]:
t

'HAUPTSTADT = Canberra STAATS- UND REGIERUNGSFORM = Bundesstaat (föderaler Staat) föderale parlamentarische Monarchie STAATSOBERHAUPT = König Charles reten durch Generalgouverneur DavidHurley REGIERUNGSCHEF = PremierministerAnthony Albanese FLÄCHE = (Liste von Staaten und Territorien nach Fläche NWOHNER = 25,8 Millionen (Liste von Staaten und Territorien nach Einwohnerzahl#Liste eptember 2021) BEV-DICHTE = 3,3 BEV-ZUNAHME = +0,3 % BEV-ZUNAHME-ZEITRAUM= (September 2021)* 1,6 Billionen USD (Liste der Länder nach Bruttoinlandsprodukt ,5 Billionen USD (Liste der Länder nach Bruttoinlandsprodukt USD (Liste der Länder nach Bruttoinlandsprodukt pro Kopf USD (Liste der Länder nach Bruttoinlandsprodukt pro Kopf P-ERWEITERT = * Total (nominal)* Total (Kaufkraftparität KKP)* P) HDI = 0,951 (Index der menschlichen Entwicklung#Sehr hohe menschliche Entwicklung 021) WÄHRUNG = Australischer Dollar (AUD) GRÜNDUNG = uar 1901 UNABHÄNGIGKEIT = tember ober 1942(Statut von z 1986(Australia Act) NATIONALHYM